In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# デバイスの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CNNの定義
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# データの準備
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# モデルと最適化関数の初期化
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)  # 学習率を増やしてみる

# 予測関数
def get_prediction(model, image):
    model.eval()
    with torch.no_grad():
        output = model(image)
        probabilities = torch.nn.functional.softmax(output, dim=1)
    pred = torch.argmax(output, dim=1)
    return pred.item(), probabilities.max().item()

# FGSMでの敵対的サンプル生成（epsilon を 0.3 に増加）
def generate_adversarial_example(model, image, label, epsilon=0.3):
    image.requires_grad = True
    output = model(image)
    loss = criterion(output, label)
    model.zero_grad()
    loss.backward()

    # デバッグ：勾配の確認
    if image.grad is None:
        print("勾配が計算されていません！")
        return None, None, None

    # ノイズ生成
    perturbation = epsilon * image.grad.sign()
    adversarial_image = image + perturbation
    adversarial_image = torch.clamp(adversarial_image, 0, 1)

    adversarial_pred, adversarial_conf = get_prediction(model, adversarial_image)
    return adversarial_image, perturbation, adversarial_pred, adversarial_conf

# 10回試行して攻撃成功回数をカウント
success_count = 0
attempts = 10

for i, (image, label) in enumerate(test_loader):
    if i >= attempts:
        break  # 10回試行で終了

    image, label = image.to(device), label.to(device)
    
    # 元の予測と信頼度
    original_pred, original_conf = get_prediction(model, image)

    # 敵対的サンプル生成
    adversarial_image, perturbation, adversarial_pred, adversarial_conf = generate_adversarial_example(model, image, label)

    # 攻撃成功判定
    if adversarial_image is not None and original_pred != adversarial_pred:
        success_count += 1
        print(f"試行 {i+1}: 成功！元の予測: {original_pred} ({original_conf:.2f}) → 敵対的サンプルの予測: {adversarial_pred} ({adversarial_conf:.2f})")
    else:
        print(f"試行 {i+1}: 失敗 元の予測: {original_pred} ({original_conf:.2f}) → 敵対的サンプルの予測: {adversarial_pred} ({adversarial_conf:.2f})")

# 攻撃成功回数を表示
print(f"\nFGSM攻撃の成功回数: {success_count} / {attempts}")
